In [203]:
import pandas as pd
import plotly.express as px

In [222]:
test = pd.read_csv('raif_test.csv')
train = pd.read_csv('raif_train.csv')

In [148]:
train['price_type'].value_counts()

0    275299
1      4493
Name: price_type, dtype: int64

In [149]:
train.head(10)

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0
5,Самара,NaN,COL_5,53.257900,50.203632,0,18,38,50,0,...,11.644737,15.823529,1875.819277,1550.818182,Самарская область,308.0,S24380,2020-01-05,100,0
6,Лаишево,NaN,COL_6,55.412357,49.561718,0,0,0,0,0,...,2.500000,2.750000,2005.500000,2013.750000,Татарстан,130.0,S14468,2020-01-05,10,0
7,Саратов,NaN,COL_7,51.521416,46.016453,0,38,117,219,0,...,6.118012,5.934783,1961.305882,1961.312500,Саратовская область,291.0,S8394,2020-01-05,10,0
8,Краснодар,NaN,COL_8,45.033280,38.976410,3,92,256,376,0,...,1.857407,1.724359,1925.483456,1921.840764,Краснодарский край,680.0,S33225,2020-01-05,100,0
9,Екатеринбург,NaN,COL_9,56.833660,60.635742,5,66,138,249,0,...,5.925676,6.125000,1967.295302,1966.200000,Свердловская область,28.0,S5897,2020-01-05,100,0


In [205]:
train = train.loc[train['price_type'] == 1]

In [151]:
train.head(10)

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
275299,Красноярск,NaN,COL_62,56.063615,92.958428,0,7,14,26,0,...,7.136842,7.612903,1976.684211,1980.483871,Красноярский край,106.351104,S5845,2020-01-05,110,1
275300,Саратов,NaN,COL_71,51.534581,46.020549,13,198,345,462,0,...,4.144608,4.164557,1940.591549,1939.348837,Саратовская область,175.039602,S27968,2020-01-05,10,1
275301,Красноярск,NaN,COL_140,56.026884,92.818323,3,15,23,33,0,...,6.851429,5.742424,1976.714286,1972.878788,Красноярский край,89.534921,S23597,2020-01-05,10,1
275302,Иркутск,NaN,COL_202,52.275528,104.251444,0,10,26,40,0,...,4.597222,5.469388,1968.098131,1975.530612,Иркутская область,419.557752,S10587,2020-01-05,10,1
275303,Белгород,NaN,COL_207,50.576545,36.584197,4,48,73,92,0,...,9.157480,9.088235,1983.125984,1980.470588,Белгородская область,1098.703185,S11220,2020-01-05,10,1
275304,Санкт-Петербург,2.0,COL_265,59.930333,30.313182,18,430,731,1160,0,...,4.579487,4.579710,1850.051282,1836.250000,Санкт-Петербург,104.202082,S6752,2020-01-05,10,1
275305,Калуга,NaN,COL_280,54.527172,36.295058,4,20,41,84,0,...,4.200000,3.269231,1968.046154,1971.960000,Калужская область,1376.836181,S6703,2020-01-05,10,1
275306,Сургут,NaN,COL_358,61.251327,73.371992,0,27,77,125,0,...,6.117117,5.459459,1983.294643,1981.189189,Ханты-Мансийский АО,338.777904,S27593,2020-01-05,10,1
275307,Иркутск,NaN,COL_430,52.264115,104.331337,0,7,27,52,0,...,6.367021,4.562500,1982.227027,1981.516129,Иркутская область,207.530443,S19643,2020-01-05,10,1
275308,Иркутск,NaN,COL_470,52.345692,104.224711,0,5,6,10,0,...,5.113208,6.767442,1971.810458,1982.527778,Иркутская область,130.964144,S28820,2020-01-05,10,1


In [152]:
train.columns

Index(['city', 'floor', 'id', 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_name', 'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in

In [153]:
fig = px.histogram(train['city'])
fig.show()

In [154]:
fig = px.histogram(test['city'])
fig.show()

In [155]:
train['city'].value_counts()

Красноярск                 543
Санкт-Петербург            374
Иркутск                    358
Томск                      227
Екатеринбург               193
                          ... 
Павловский Посад             1
Коркино                      1
Белово                       1
рабочий посёлок Маркова      1
Клинцы                       1
Name: city, Length: 190, dtype: int64

In [156]:
test['city'].value_counts()

Красноярск           405
Иркутск              228
Санкт-Петербург      205
Томск                138
Петрозаводск         136
                    ... 
Каменск-Уральский      1
Уссурийск              1
Набережные Челны       1
Руза                   1
Минусинск              1
Name: city, Length: 165, dtype: int64

In [206]:
# заменим тип недвижимости с чисел на буквы
tmp_set = set(train['realty_type'].values)
for i, element in enumerate(tmp_set):
    train['realty_type'].replace(element, f'type_{i}', inplace=True)

In [158]:
train['realty_type'].value_counts()

type_0    2210
type_1    1330
type_2     953
Name: realty_type, dtype: int64

In [159]:
train.head(10)

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
275299,Красноярск,NaN,COL_62,56.063615,92.958428,0,7,14,26,0,...,7.136842,7.612903,1976.684211,1980.483871,Красноярский край,106.351104,S5845,2020-01-05,type_2,1
275300,Саратов,NaN,COL_71,51.534581,46.020549,13,198,345,462,0,...,4.144608,4.164557,1940.591549,1939.348837,Саратовская область,175.039602,S27968,2020-01-05,type_0,1
275301,Красноярск,NaN,COL_140,56.026884,92.818323,3,15,23,33,0,...,6.851429,5.742424,1976.714286,1972.878788,Красноярский край,89.534921,S23597,2020-01-05,type_0,1
275302,Иркутск,NaN,COL_202,52.275528,104.251444,0,10,26,40,0,...,4.597222,5.469388,1968.098131,1975.530612,Иркутская область,419.557752,S10587,2020-01-05,type_0,1
275303,Белгород,NaN,COL_207,50.576545,36.584197,4,48,73,92,0,...,9.157480,9.088235,1983.125984,1980.470588,Белгородская область,1098.703185,S11220,2020-01-05,type_0,1
275304,Санкт-Петербург,2.0,COL_265,59.930333,30.313182,18,430,731,1160,0,...,4.579487,4.579710,1850.051282,1836.250000,Санкт-Петербург,104.202082,S6752,2020-01-05,type_0,1
275305,Калуга,NaN,COL_280,54.527172,36.295058,4,20,41,84,0,...,4.200000,3.269231,1968.046154,1971.960000,Калужская область,1376.836181,S6703,2020-01-05,type_0,1
275306,Сургут,NaN,COL_358,61.251327,73.371992,0,27,77,125,0,...,6.117117,5.459459,1983.294643,1981.189189,Ханты-Мансийский АО,338.777904,S27593,2020-01-05,type_0,1
275307,Иркутск,NaN,COL_430,52.264115,104.331337,0,7,27,52,0,...,6.367021,4.562500,1982.227027,1981.516129,Иркутская область,207.530443,S19643,2020-01-05,type_0,1
275308,Иркутск,NaN,COL_470,52.345692,104.224711,0,5,6,10,0,...,5.113208,6.767442,1971.810458,1982.527778,Иркутская область,130.964144,S28820,2020-01-05,type_0,1


In [160]:
first_floor_data = train.loc[(train['floor'] == '1.0') | (train['floor'] == '1') | (train['floor'] == 1.0)]

In [161]:
first_floor_data['realty_type'].value_counts()

type_0    551
type_2    359
type_1    279
Name: realty_type, dtype: int64

Не нашелся паттенр, ну штош

In [162]:
train.fillna(0, inplace=True)

In [163]:
train['floor'].value_counts()

0                         2464
1.0                        747
1                          442
2.0                        135
-1.0                       121
                          ... 
подвал, цоколь, 1 этаж       1
мезонин                      1
подвал, 1,2,3                1
1-й, 2-й                     1
18                           1
Name: floor, Length: 89, dtype: int64

In [165]:
train = train.loc[train['floor'].astype(str).str.isnumeric()]

In [167]:
train['floor'].value_counts()

0     2464
1      442
2       74
3       49
5       27
4       22
6       17
7        6
29       3
8        3
13       3
36       2
10       2
12       1
11       1
9        1
15       1
18       1
Name: floor, dtype: int64

In [93]:
prik = train['floor'].value_counts()

# ВОТ ЗДЕСЬ НАЧИНЕТСЯ ПРОТОПТИП

In [207]:
train['floor'].fillna('unknown', inplace=True)

In [208]:
train.fillna(0, inplace=True)

In [209]:
target = train['per_square_meter_price']
train.drop(['id', 'lat', 'lng', 'street', 'date', 'per_square_meter_price'], axis=1, inplace=True)

In [210]:
from catboost import CatBoostRegressor

In [211]:
model = CatBoostRegressor(iterations=1000,
                          verbose=False,
                          loss_function='RMSE',
                          cat_features=['city', 'region', 'realty_type', 'floor', 'osm_city_nearest_name'])

In [212]:
model.fit(train, target)

In [223]:
test_id = test['id']
test.drop(['id', 'lat', 'lng', 'street', 'date'], axis=1, inplace=True)
test['floor'].fillna('unknown', inplace=True)
test.fillna(0, inplace=True)

In [224]:
predictions = model.predict(test)

In [225]:
predictions

array([49228.88818823, 53370.60754086, 51028.6938068 , ...,
       61126.17804102, 80394.7358747 , 58688.91180821])

In [227]:
result = pd.DataFrame({'id': test_id, 'per_square_meter_price': predictions})

In [229]:
result.to_csv('predictions.csv')